In [1]:
import os
import pandas as pd 
import seaborn as sns                                                   
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.neural_network import MLPClassifier
import numpy as np
from xgboost import XGBClassifier, plot_importance
import pickle


In [2]:
import pandas as pd

# Ruta del archivo
file_path = r"C:\Users\Jose\Desktop\IA\train.csv"

# Cargar el archivo CSV
try:
    data = pd.read_csv(file_path)
    # Mostrar las primeras filas
    print("Archivo cargado")
except FileNotFoundError:
    print("No se encontró el archivo. Verifica la ruta.")
except Exception as e:
    print(f"Ocurrió un error: {e}")


Archivo cargado


In [3]:
data

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO,RENDIMIENTO_GLOBAL
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Postgrado,No,medio-alto
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Técnica o tecnológica incompleta,No,bajo
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,No,bajo
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Secundaria (Bachillerato) completa,No,alto
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Primaria completa,No,medio-bajo
...,...,...,...,...,...,...,...,...,...,...,...,...
692495,25096,20195,BIOLOGIA,LA GUAJIRA,Entre 500 mil y menos de 1 millón,Entre 11 y 20 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) incompleta,Si,medio-alto
692496,754213,20212,PSICOLOGIA,NORTE SANTANDER,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Primaria incompleta,Secundaria (Bachillerato) incompleta,No,bajo
692497,504185,20183,ADMINISTRACIÓN EN SALUD OCUPACIONAL,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,Menos de 10 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) incompleta,Si,medio-bajo
692498,986620,20195,PSICOLOGIA,TOLIMA,Entre 2.5 millones y menos de 4 millones,Menos de 10 horas,Estrato 1,No,Primaria completa,Primaria completa,Si,bajo


In [4]:
# Verificar la cantidad de datos nulos por columna
print(data.isnull().sum())


ID                                    0
PERIODO                               0
ESTU_PRGM_ACADEMICO                   0
ESTU_PRGM_DEPARTAMENTO                0
ESTU_VALORMATRICULAUNIVERSIDAD     6287
ESTU_HORASSEMANATRABAJA           30857
FAMI_ESTRATOVIVIENDA              32137
FAMI_TIENEINTERNET                26629
FAMI_EDUCACIONPADRE               23178
FAMI_EDUCACIONMADRE               23664
ESTU_PAGOMATRICULAPROPIO           6498
RENDIMIENTO_GLOBAL                    0
dtype: int64


In [5]:
import pandas as pd

def preprocess_data(data):
    """
    Preprocesa un DataFrame aplicando diversas transformaciones:
    - Elimina columnas irrelevantes.
    - Codifica los programas académicos más frecuentes como 'CARRERA_TOP_1', 'CARRERA_TOP_2', etc.
    - Realiza mapeos personalizados y llena valores nulos con la media.

    Args:
        data (pd.DataFrame): El DataFrame original.

    Returns:
        pd.DataFrame: El DataFrame procesado.
    """
    # Eliminar la columna 'Unnamed: 0' si existe
    if 'Unnamed: 0' in data.columns:
        data = data.drop(columns=['Unnamed: 0'])
    
    # Eliminar la columna 'PERIODO' si existe
    if 'PERIODO' in data.columns:
        data = data.drop(columns=['PERIODO'])

    # Top 20 programas más frecuentes en 'ESTU_PRGM_ACADEMICO' y etiquetar el resto como 'Otros'
    top_20_programas = data['ESTU_PRGM_ACADEMICO'].value_counts().head(20).index
    data['ESTU_PRGM_ACADEMICO'] = data['ESTU_PRGM_ACADEMICO'].apply(
        lambda x: x if x in top_20_programas else 'Otros'
    )
    
    # Realizar one-hot encoding en 'ESTU_PRGM_ACADEMICO'
    dummies = pd.get_dummies(data['ESTU_PRGM_ACADEMICO'], prefix='CARRERA_TOP')

    # Renombrar las columnas con un índice enumerado
    dummies.columns = [f'CARRERA_TOP_{i+1}' for i in range(len(dummies.columns))]

    # Añadir las columnas de one-hot encoding al DataFrame original y eliminar la columna original
    data = pd.concat([data.drop(columns=['ESTU_PRGM_ACADEMICO']), dummies], axis=1)

    # Eliminar la columna 'CARRERA_TOP_N' correspondiente a 'Otros' si existe
    if 'CARRERA_TOP_20' in data.columns:  # Supone que 'Otros' es la última columna si existiera
        data = data.drop(columns=['CARRERA_TOP_20'])

    # Mapeo para 'ESTU_PRGM_DEPARTAMENTO'
    departamento_dict = {
        'ANTIOQUIA': 4, 'ARAUCA': 1, 'ATLANTICO': 3, 'BOGOTÁ': 3, 'BOLIVAR': 2,
        'BOYACA': 5, 'CALDAS': 5, 'CAQUETA': 0, 'CASANARE': 0, 'CAUCA': 0,
        'CESAR': 0, 'CHOCO': 0, 'CORDOBA': 0, 'CUNDINAMARCA': 5, 'HUILA': 0,
        'LA GUAJIRA': 0, 'MAGDALENA': 3, 'META': 3, 'NARIÑO': 2,
        'NORTE SANTANDER': 2, 'PUTUMAYO': 1, 'QUINDIO': 4, 'RISARALDA': 4,
        'SANTANDER': 5, 'SUCRE': 1, 'TOLIMA': 2, 'VALLE': 4, 'AMAZONAS': 0,
        'SAN ANDRES': 0
    }
    data['ESTU_PRGM_DEPARTAMENTO'] = data['ESTU_PRGM_DEPARTAMENTO'].map(departamento_dict)

    # Mapeos personalizados para otras columnas
    horas_trabajo_dict = {
        'Menos de 10 horas': 1, '0': 0, 'Entre 21 y 30 horas': 3,
        'Entre 11 y 20 horas': 2, 'Más de 30 horas': 4
    }
    estrato_vivienda_dict = {
        'Estrato 3': 3, 'Estrato 4': 4, 'Estrato 2': 2, 'Estrato 1': 1,
        'Estrato 5': 5, 'Estrato 6': 6, 'Sin Estrato': 0
    }
    internet_dict = {'Si': 1, 'No': 0}
    educacion_padre_dict = {
        'Ninguno': 0, 'Primaria incompleta': 1, 'Primaria completa': 2,
        'Secundaria (Bachillerato) incompleta': 3, 'Secundaria (Bachillerato) completa': 4,
        'Técnica o tecnológica incompleta': 5, 'Técnica o tecnológica completa': 6,
        'Educación profesional incompleta': 7, 'Educación profesional completa': 8,
        'Postgrado': 9, 'No sabe': 10, 'No Aplica': 11
    }
    educacion_madre_dict = educacion_padre_dict.copy()
    pago_matricula_dict = {'No': 0, 'Si': 1}
    value_mapping = {
        'Entre 5.5 millones y menos de 7 millones': 7,
        'Entre 2.5 millones y menos de 4 millones': 4,
        'Entre 4 millones y menos de 5.5 millones': 5,
        'Entre 1 millón y menos de 2.5 millones': 2,
        'Entre 500 mil y menos de 1 millón': 1,
        'Menos de 500 mil': 0,
        'Más de 7 millones': 8,
        'No pagó matrícula': -1
    }

    # Aplicar los mapeos
    data['ESTU_HORASSEMANATRABAJA'] = data['ESTU_HORASSEMANATRABAJA'].map(horas_trabajo_dict)
    data['FAMI_ESTRATOVIVIENDA'] = data['FAMI_ESTRATOVIVIENDA'].map(estrato_vivienda_dict)
    data['FAMI_TIENEINTERNET'] = data['FAMI_TIENEINTERNET'].map(internet_dict)
    data['FAMI_EDUCACIONPADRE'] = data['FAMI_EDUCACIONPADRE'].map(educacion_padre_dict)
    data['FAMI_EDUCACIONMADRE'] = data['FAMI_EDUCACIONMADRE'].map(educacion_madre_dict)
    data['ESTU_PAGOMATRICULAPROPIO'] = data['ESTU_PAGOMATRICULAPROPIO'].map(pago_matricula_dict)
    data['ESTU_VALORMATRICULAUNIVERSIDAD'] = data['ESTU_VALORMATRICULAUNIVERSIDAD'].map(value_mapping)

    # Llenar valores nulos con la media
    data = data.fillna(data.mean(numeric_only=True))

    # Convertir True/False a 1/0
    data = data.replace({True: 1, False: 0})

    return data


In [6]:

# Aplica la función preprocess_data
processed_data = preprocess_data(data)


C:\Users\Jose\AppData\Local\Temp\ipykernel_26188\2379587069.py:98: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data = data.replace({True: 1, False: 0})


In [7]:
# Ver las columnas del DataFrame procesado
processed_data.columns


Index(['ID', 'ESTU_PRGM_DEPARTAMENTO', 'ESTU_VALORMATRICULAUNIVERSIDAD',
       'ESTU_HORASSEMANATRABAJA', 'FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEINTERNET',
       'FAMI_EDUCACIONPADRE', 'FAMI_EDUCACIONMADRE',
       'ESTU_PAGOMATRICULAPROPIO', 'RENDIMIENTO_GLOBAL', 'CARRERA_TOP_1',
       'CARRERA_TOP_2', 'CARRERA_TOP_3', 'CARRERA_TOP_4', 'CARRERA_TOP_5',
       'CARRERA_TOP_6', 'CARRERA_TOP_7', 'CARRERA_TOP_8', 'CARRERA_TOP_9',
       'CARRERA_TOP_10', 'CARRERA_TOP_11', 'CARRERA_TOP_12', 'CARRERA_TOP_13',
       'CARRERA_TOP_14', 'CARRERA_TOP_15', 'CARRERA_TOP_16', 'CARRERA_TOP_17',
       'CARRERA_TOP_18', 'CARRERA_TOP_19', 'CARRERA_TOP_21'],
      dtype='object')

In [8]:
# Verificar la cantidad de datos nulos por columna
print(processed_data.isnull().sum())


ID                                0
ESTU_PRGM_DEPARTAMENTO            0
ESTU_VALORMATRICULAUNIVERSIDAD    0
ESTU_HORASSEMANATRABAJA           0
FAMI_ESTRATOVIVIENDA              0
FAMI_TIENEINTERNET                0
FAMI_EDUCACIONPADRE               0
FAMI_EDUCACIONMADRE               0
ESTU_PAGOMATRICULAPROPIO          0
RENDIMIENTO_GLOBAL                0
CARRERA_TOP_1                     0
CARRERA_TOP_2                     0
CARRERA_TOP_3                     0
CARRERA_TOP_4                     0
CARRERA_TOP_5                     0
CARRERA_TOP_6                     0
CARRERA_TOP_7                     0
CARRERA_TOP_8                     0
CARRERA_TOP_9                     0
CARRERA_TOP_10                    0
CARRERA_TOP_11                    0
CARRERA_TOP_12                    0
CARRERA_TOP_13                    0
CARRERA_TOP_14                    0
CARRERA_TOP_15                    0
CARRERA_TOP_16                    0
CARRERA_TOP_17                    0
CARRERA_TOP_18              

In [9]:
# Guardar el DataFrame 'processed_data' en un archivo CSV
processed_data.to_csv('processed_datatrain.csv', index=False)

In [10]:
import pandas as pd

# Ruta del archivo
file_path = r"C:\Users\Jose\Desktop\IA\test.csv"

# Cargar el archivo CSV
try:
    datatest = pd.read_csv(file_path)
    # Mostrar las primeras filas
    print("Archivo cargado")
except FileNotFoundError:
    print("No se encontró el archivo. Verifica la ruta.")
except Exception as e:
    print(f"Ocurrió un error: {e}")


Archivo cargado


In [11]:
datatest

,Unnamed: 0,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO
0,0,550236,20183,TRABAJO SOCIAL,BOLIVAR,Menos de 500 mil,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica completa,Primaria completa,Si
1,1,98545,20203,ADMINISTRACION COMERCIAL Y DE MERCADEO,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Técnica o tecnológica completa,No
2,2,499179,20212,INGENIERIA MECATRONICA,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,0,Estrato 3,Si,Secundaria (Bachillerato) incompleta,Secundaria (Bachillerato) completa,No
3,3,782980,20195,CONTADURIA PUBLICA,SUCRE,Entre 1 millón y menos de 2.5 millones,Entre 21 y 30 horas,Estrato 1,No,Primaria incompleta,Primaria incompleta,No
4,4,785185,20212,ADMINISTRACION DE EMPRESAS,ATLANTICO,Entre 2.5 millones y menos de 4 millones,Entre 11 y 20 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,No
...,...,...,...,...,...,...,...,...,...,...,...,...
296781,296781,496981,20195,ADMINISTRACION DE EMPRESAS,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 1,Si,Primaria incompleta,Primaria incompleta,Si
296782,296782,209415,20183,DERECHO,META,Entre 1 millón y menos de 2.5 millones,0,Estrato 4,Si,Educación profesional completa,Educación profesional completa,No
296783,296783,239074,20212,DERECHO,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Educación profesional completa,No
296784,296784,963852,20195,INGENIERIA AERONAUTICA,ANTIOQUIA,Entre 5.5 millones y menos de 7 millones,Entre 11 y 20 horas,Estrato 3,Si,Educación profesional completa,Educación profesional completa,No


In [12]:

# Aplica la función preprocess_data
processed_datatest = preprocess_data(datatest)


C:\Users\Jose\AppData\Local\Temp\ipykernel_26188\2379587069.py:98: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data = data.replace({True: 1, False: 0})


In [13]:
processed_datatest

,ID,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO,CARRERA_TOP_1,...,CARRERA_TOP_11,CARRERA_TOP_12,CARRERA_TOP_13,CARRERA_TOP_14,CARRERA_TOP_15,CARRERA_TOP_16,CARRERA_TOP_17,CARRERA_TOP_18,CARRERA_TOP_19,CARRERA_TOP_21
0,550236,2.0,0.0,1.0,3.0,1.0,6.0,2.0,1.0,0,...,0,0,0,0,0,0,0,0,0,1
1,98545,4.0,4.0,3.0,2.0,1.0,4.0,6.0,0.0,0,...,0,0,0,0,0,0,0,1,0,0
2,499179,3.0,2.0,0.0,3.0,1.0,3.0,4.0,0.0,0,...,0,0,0,0,0,0,0,1,0,0
3,782980,1.0,2.0,3.0,1.0,0.0,1.0,1.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
4,785185,3.0,4.0,2.0,2.0,1.0,4.0,4.0,0.0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296781,496981,3.0,4.0,4.0,1.0,1.0,1.0,1.0,1.0,1,...,0,0,0,0,0,0,0,0,0,0
296782,209415,3.0,2.0,0.0,4.0,1.0,8.0,8.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
296783,239074,3.0,4.0,4.0,3.0,1.0,4.0,8.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
296784,963852,4.0,7.0,2.0,3.0,1.0,8.0,8.0,0.0,0,...,0,0,0,0,0,0,0,1,0,0


In [14]:
# Comparar nombres de columnas
train_columns = set(processed_data.columns)
test_columns = set(processed_datatest.columns)

# Columnas que están en train pero no en test
only_in_train = train_columns - test_columns

# Columnas que están en test pero no en train
only_in_test = test_columns - train_columns

print("Columnas solo en train:", only_in_train)
print("Columnas solo en test:", only_in_test)


Columnas solo en train: {'RENDIMIENTO_GLOBAL'}
Columnas solo en test: set()


In [15]:
# Ver los valores únicos de la columna RENDIMIENTO_GLOBAL
valores_unicos = processed_data['RENDIMIENTO_GLOBAL'].unique()
print(valores_unicos)


['medio-alto' 'bajo' 'alto' 'medio-bajo']


In [16]:
# Crear un diccionario de mapeo para convertir las categorías a números
rendimiento_mapping = {
    'bajo': 0,
    'medio-bajo': 1,
    'medio-alto': 2,
    'alto': 3
}

# Aplicar el mapeo a la columna RENDIMIENTO_GLOBAL
processed_data['RENDIMIENTO_GLOBAL'] = processed_data['RENDIMIENTO_GLOBAL'].map(rendimiento_mapping)

# Verificar la conversión
print(processed_data['RENDIMIENTO_GLOBAL'].unique())


[2 0 3 1]


In [17]:
from sklearn.model_selection import train_test_split

# 1. Eliminar la columna 'ID'
processed_data = processed_data.drop(columns=['ID'])

# 2. Seleccionar 'RENDIMIENTO_GLOBAL' como target
X = processed_data.drop(columns=['RENDIMIENTO_GLOBAL'])  # Características (features)
y = processed_data['RENDIMIENTO_GLOBAL']  # Objetivo (target)

# 4. Dividir los datos en conjunto de entrenamiento (80%) y prueba (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Verificar las formas de los conjuntos
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")


X_train shape: (554000, 28)
X_test shape: (138500, 28)
y_train shape: (554000,)
y_test shape: (138500,)


RANDOM FOREST GRID SEARCH

In [18]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import classification_report

# 1. Definir el modelo Random Forest
rf_model = RandomForestClassifier(random_state=42)

# 2. Configurar el Grid Search con hiperparámetros básicos
param_grid = {
    'n_estimators': [100, 200, 300],           # Número de árboles
    'max_depth': [None, 10, 20, 30],           # Profundidad máxima de cada árbol
    'min_samples_split': [2, 5, 10],           # Mínimo de muestras para dividir un nodo
    'min_samples_leaf': [1, 2, 4]              # Mínimo de muestras por hoja
}

# 3. Configurar validación cruzada con 5 K-Folds
cv = KFold(n_splits=2, shuffle=True, random_state=42)

# 4. Configurar GridSearchCV
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, 
                           cv=cv, n_jobs=-1, verbose=1, scoring='accuracy')

# 5. Entrenar el modelo con los datos de entrenamiento
grid_search.fit(X_train, y_train)

# 6. Mostrar los mejores hiperparámetros
print(f"Mejores hiperparámetros: {grid_search.best_params_}")
print(f"Mejor precisión en validación: {grid_search.best_score_:.4f}")

# 7. Evaluar el modelo en el conjunto de prueba
y_pred = grid_search.best_estimator_.predict(X_test)

# 8. Reporte de clasificación
print("\nReporte de clasificación en el conjunto de prueba:")
print(classification_report(y_test, y_pred))


Fitting 2 folds for each of 108 candidates, totalling 216 fits
Mejores hiperparámetros: {'max_depth': 20, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 300}
Mejor precisión en validación: 0.4035

Reporte de clasificación en el conjunto de prueba:
              precision    recall  f1-score   support

           0       0.42      0.53      0.47     34573
           1       0.32      0.25      0.28     34503
           2       0.31      0.23      0.26     34259
           3       0.51      0.62      0.56     35165

    accuracy                           0.41    138500
   macro avg       0.39      0.41      0.39    138500
weighted avg       0.39      0.41      0.39    138500



In [19]:
# 1. Guardar la columna 'ID' de processed_datatest antes de dropearla
id_column = processed_datatest['ID']

# 2. Eliminar la columna 'ID' de processed_datatest (si está presente)
processed_datatest = processed_datatest.drop(columns=['ID'], errors='ignore')

# 3. Asegúrate de que el orden de las columnas en processed_datatest sea el mismo que en X_train
processed_datatest = processed_datatest[X_train.columns]

# 4. Predecir los valores de RENDIMIENTO_GLOBAL para el conjunto processed_datatest
y_test_pred = grid_search.best_estimator_.predict(processed_datatest)

# 5. Crear un DataFrame con las predicciones y los IDs
output = pd.DataFrame({'ID': id_column, 'Prediccion': y_test_pred})

# 6. Visualizar el DataFrame con las predicciones
print(output.head())  # Muestra las primeras 5 filas



       ID  Prediccion
0  550236           0
1   98545           2
2  499179           3
3  782980           0
4  785185           1


In [20]:
# 7. Guardar el resultado en un archivo CSV
output.to_csv('predicciones_processed_datatest.csv', index=False)

# 8. Imprimir mensaje de confirmación
print("Predicciones guardadas en 'predicciones_processed_datatest.csv'")


Predicciones guardadas en 'predicciones_processed_datatest.csv'


In [21]:
output

,ID,Prediccion
0,550236,0
1,98545,2
2,499179,3
3,782980,0
4,785185,1
...,...,...
296781,496981,0
296782,209415,3
296783,239074,2
296784,963852,3


In [22]:
import pandas as pd

# Supongamos que 'y_test_pred' contiene las predicciones (0-3)
# Invertimos el mapeo
inverted_mapping = {0: 'bajo', 1: 'medio-bajo', 2: 'medio-alto', 3: 'alto'}

# Aplicamos el mapeo invertido a las predicciones
y_test_pred_inverted = [inverted_mapping[val] for val in y_test_pred]

# Crear un DataFrame con los ID y las predicciones invertidas
output_inverted = pd.DataFrame({'ID': id_column, 'Prediccion': y_test_pred_inverted})

# Guardar el DataFrame en un archivo CSV
output_inverted.to_csv('predicciones_invertidas.csv', index=False)

# Verificar las primeras filas
print(output_inverted.head())

# Confirmar que el archivo fue guardado
print("Predicciones invertidas guardadas en 'predicciones_invertidas.csv'")


       ID  Prediccion
0  550236        bajo
1   98545  medio-alto
2  499179        alto
3  782980        bajo
4  785185  medio-bajo
Predicciones invertidas guardadas en 'predicciones_invertidas.csv'


In [23]:
output_inverted

,ID,Prediccion
0,550236,bajo
1,98545,medio-alto
2,499179,alto
3,782980,bajo
4,785185,medio-bajo
...,...,...
296781,496981,bajo
296782,209415,alto
296783,239074,medio-alto
296784,963852,alto


In [24]:
import os
import pandas as pd 
import seaborn as sns                                                   
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from xgboost import XGBClassifier
from xgboost import plot_importance
import pickle

# 1. Definir el modelo XGBoost con parámetros básicos
xgb_model = XGBClassifier(
    n_estimators=100,           # Número de árboles
    learning_rate=0.1,          # Tasa de aprendizaje
    max_depth=6,                # Profundidad máxima de los árboles
    subsample=0.8,              # Submuestras para cada árbol
    colsample_bytree=0.8,       # Fracción de columnas usadas por árbol
    random_state=42,            # Semilla aleatoria
    use_label_encoder=False,    # Evitar warnings
    eval_metric='logloss'       # Métrica de evaluación
)

# 2. Entrenar el modelo con los datos de entrenamiento
xgb_model.fit(X_train, y_train)

# 3. Realizar predicciones en el conjunto de prueba
y_pred = xgb_model.predict(X_test)

# 4. Evaluar el modelo: Matriz de Confusión y Reporte de Clasificación
print("\nMatriz de confusión:")
print(confusion_matrix(y_test, y_pred))

print("\nReporte de clasificación en el conjunto de prueba:")
print(classification_report(y_test, y_pred))



C:\Users\Jose\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\xgboost\core.py:158: UserWarning: [11:01:30] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



Matriz de confusión:
[[18838  7184  4548  4003]
 [13301  8424  6444  6334]
 [ 8525  7345  7610 10779]
 [ 3927  3566  5932 21740]]

Reporte de clasificación en el conjunto de prueba:
              precision    recall  f1-score   support

           0       0.42      0.54      0.48     34573
           1       0.32      0.24      0.28     34503
           2       0.31      0.22      0.26     34259
           3       0.51      0.62      0.56     35165

    accuracy                           0.41    138500
   macro avg       0.39      0.41      0.39    138500
weighted avg       0.39      0.41      0.39    138500



In [25]:


# 3. Asegúrate de que el orden de las columnas en processed_datatest sea el mismo que en X_train
processed_datatest = processed_datatest[X_train.columns]

# 4. Predecir los valores de RENDIMIENTO_GLOBAL para el conjunto processed_datatest usando el modelo entrenado
y_test_pred = xgb_model.predict(processed_datatest)

# 5. Crear un DataFrame con las predicciones y los IDs
output = pd.DataFrame({'ID': id_column, 'Prediccion': y_test_pred})

# 6. Visualizar el DataFrame con las predicciones
print(output.head())  # Muestra las primeras 5 filas

# 7. (Opcional) Guardar el DataFrame con las predicciones en un archivo CSV
output.to_csv('predicciones_xgbNGS.csv', index=False)
print("\nPredicciones guardadas en 'predicciones_xgb.csv'")


       ID  Prediccion
0  550236           0
1   98545           2
2  499179           1
3  782980           0
4  785185           1

Predicciones guardadas en 'predicciones_xgb.csv'


In [ ]:

import os
import pandas as pd 
import seaborn as sns                                                   
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, RandomizedSearchCV
import numpy as np
from sklearn.metrics import confusion_matrix
from xgboost import XGBClassifier
from xgboost import plot_importance
import pickle

# Definir los parámetros para la búsqueda aleatoria
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss')

param_dist = {
    'n_estimators': np.arange(50, 300, 200),
    'learning_rate': [0.0001, 0.001, 0.1, 0.2],
    'max_depth': np.arange(3, 150),
    'subsample': np.linspace(0.5, 1.0, 15),
    'colsample_bytree': np.linspace(0.5, 1.0, 15)
}

# Configurar RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=xgb,
                                    param_distributions=param_dist,
                                    n_iter=200,
                                    cv=5,
                                    n_jobs=-1,
                                    verbose=5,
                                    random_state=42,
                                    return_train_score=True,
                                    scoring=['accuracy','precision'],
                                    refit='accuracy')

# Realizar la búsqueda
random_search.fit(X_train, y_train)

# Imprimir los mejores parámetros y la mejor puntuación
print("Mejores parámetros encontrados por RandomizedSearchCV:")
print(random_search.best_params_)
print("Mejor puntuación encontrada por RandomizedSearchCV:")
print(random_search.best_score_)

Fitting 5 folds for each of 200 candidates, totalling 1000 fits


In [ ]:
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report
from xgboost import XGBClassifier
import pickle

# Definir el mapeo invertido
inverted_mapping = {0: 'bajo', 1: 'medio-bajo', 2: 'medio-alto', 3: 'alto'}

# 1. Obtener los mejores parámetros del random_search
best_params = random_search.best_params_

# 2. Crear un nuevo modelo XGBoost con los mejores parámetros encontrados
xgb_model = XGBClassifier(
    n_estimators=best_params['n_estimators'],
    learning_rate=best_params['learning_rate'],
    max_depth=best_params['max_depth'],
    subsample=best_params['subsample'],
    colsample_bytree=best_params['colsample_bytree'],
    use_label_encoder=False,
    eval_metric='logloss'
)

# 3. Entrenar el modelo con los datos de entrenamiento si no se ha hecho previamente
xgb_model.fit(X_train, y_train)

# 4. Asegúrate de que el conjunto de datos de prueba tenga las mismas columnas que los datos de entrenamiento
processed_datatest = processed_datatest[X_train.columns]

# 5. Realizar predicciones en el conjunto de datos processed_datatest
y_test_pred = xgb_model.predict(processed_datatest)

# 6. Invertir las predicciones usando el mapeo
y_test_pred_mapped = pd.Series(y_test_pred).map(inverted_mapping)

# 7. Crear un DataFrame con las predicciones y los IDs (suponiendo que 'id_column' es la columna de IDs)
output = pd.DataFrame({'ID': id_column, 'Prediccion': y_test_pred_mapped})

# 8. Visualizar el DataFrame con las predicciones
print(output.head())  # Muestra las primeras 5 filas

# 9. (Opcional) Guardar el DataFrame con las predicciones en un archivo CSV
output.to_csv('predicciones_xgb.csv', index=False)
print("\nPredicciones guardadas en 'predicciones_xgb.csv'")



       ID  Prediccion
0  550236        bajo
1   98545  medio-alto
2  499179        alto
3  782980        bajo
4  785185  medio-alto

Predicciones guardadas en 'predicciones_xgb.csv'


RED NEURONAL

In [ ]:
import os
import pandas as pd 
import seaborn as sns                                                   
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.neural_network import MLPClassifier
import numpy as np
from sklearn.metrics import confusion_matrix
import pickle
param_distributions = {
    'hidden_layer_sizes': [(4,2),(5,2), (2,2), (10,10), (32,32), (8,8)],  # probar diferentes arquitecturas de capas ocultas
    'activation': ['logistic', 'relu'],  # función de activación
    'solver': ['sgd', 'adam'],  # algoritmo de optimización
    'learning_rate': ['constant','adaptive'],  # tasa de aprendizaje
}
mlp = MLPClassifier(max_iter=100)
random_search = RandomizedSearchCV(estimator=mlp,
                                    param_distributions=param_distributions,
                                    n_iter=25,
                                    cv=5,
                                    n_jobs=-1,
                                    verbose=5,
                                    random_state=42,
                                    return_train_score=True,
                                    scoring=['accuracy','precision'],
                                    refit='accuracy')


random_search.fit(X_train, y_train)
print("Mejores parámetros encontrados por RandomizedSearchCV:")
print(random_search.best_params_)
print("Mejor puntuación encontrada por RandomizedSearchCV:")
print(random_search.best_score_)

Fitting 5 folds for each of 25 candidates, totalling 125 fits


c:\Users\Daniel\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_search.py:1051: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan]
  warnings.warn(
c:\Users\Daniel\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_search.py:1051: UserWarning: One or more of the train scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan]
  warnings.warn(


Mejores parámetros encontrados por RandomizedSearchCV:
{'solver': 'adam', 'learning_rate': 'constant', 'hidden_layer_sizes': (10, 10), 'activation': 'relu'}
Mejor puntuación encontrada por RandomizedSearchCV:
0.3925992779783393


c:\Users\Daniel\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


In [ ]:
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.neural_network import MLPClassifier
import pickle

# Definir el mapeo invertido
inverted_mapping = {0: 'bajo', 1: 'medio-bajo', 2: 'medio-alto', 3: 'alto'}

# 1. Obtener los mejores parámetros del random_search
best_params = random_search.best_params_

# 2. Crear un nuevo modelo MLP con los mejores parámetros encontrados
mlp_model = MLPClassifier(
    hidden_layer_sizes=best_params['hidden_layer_sizes'],
    activation=best_params['activation'],
    solver=best_params['solver'],
    learning_rate=best_params['learning_rate'],
    max_iter=100,  # Este es el número máximo de iteraciones de entrenamiento
    random_state=42
)

# 3. Entrenar el modelo con los datos de entrenamiento si no se ha hecho previamente
mlp_model.fit(X_train, y_train)

# 4. Asegúrate de que el conjunto de datos de prueba tenga las mismas columnas que los datos de entrenamiento
processed_datatest = processed_datatest[X_train.columns]

# 5. Realizar predicciones en el conjunto de datos processed_datatest
y_test_pred = mlp_model.predict(processed_datatest)

# 6. Invertir las predicciones usando el mapeo
y_test_pred_mapped = pd.Series(y_test_pred).map(inverted_mapping)

# 7. Crear un DataFrame con las predicciones y los IDs (suponiendo que 'id_column' es la columna de IDs)
output = pd.DataFrame({'ID': id_column, 'Prediccion': y_test_pred_mapped})

# 8. Visualizar el DataFrame con las predicciones
print(output.head())  # Muestra las primeras 5 filas

# 9. (Opcional) Guardar el DataFrame con las predicciones en un archivo CSV
output.to_csv('predicciones_mlp.csv', index=False)
print("\nPredicciones guardadas en 'predicciones_mlp.csv'")



c:\Users\Daniel\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


       ID  Prediccion
0  550236        bajo
1   98545  medio-alto
2  499179  medio-alto
3  782980        bajo
4  785185  medio-bajo

Predicciones guardadas en 'predicciones_mlp.csv'
